In [1]:
!pip install pyspark

In [26]:
!pip install psycopg2

In [4]:
from pyspark.sql import SparkSession
json_path="AmazonProduct.json"
spark = SparkSession.builder.appName("JSON to CSV").getOrCreate()
json_data = spark.read.json(json_path)

In [5]:
header = json_data.schema.names
header

['about_product',
 'actual_price',
 'category',
 'discount_percentage',
 'discounted_price',
 'img_link',
 'product_id',
 'product_link',
 'product_name',
 'rating',
 'rating_count',
 'review_content',
 'review_id',
 'review_title',
 'user_id',
 'user_name']

In [13]:
# Rename the columns
df = json_data.toDF(*header)

In [ ]:
df.write.csv('AmazonProduct.csv')

In [19]:
csv_data = spark.read.csv("AmazonProduct.csv",header=True)
csv_data.show()

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|        product_name|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B07JW9H4J1|Wayona Nylon Brai...|Computers&Accesso...|            ₹399|      ₹1,099|                64%|   4.2|      2

In [20]:
# Count the number of rows in the DataFrame
row_count = csv_data.count()

# Print the row count
print("Number of rows: ", row_count)

Number of rows:  1465


In [29]:
import psycopg2
import csv

In [68]:
conn = psycopg2.connect(
    host="localhost",
    database="AmazonProduct",
    user="postgres",
    password="Alpha@123"
)


In [38]:
# Create a cursor object to interact with the database
cur = conn.cursor()

# Define the table name and CSV file path
table_name = 'ProductRecommendation'
csv_file_path = 'AmazonProduct.csv'

# Read the CSV file and extract the column names and data types
with open(csv_file_path, 'r', newline='',encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader)  # Get the header row
    types = ['VARCHAR' for _ in header]  # Assuming all columns as VARCHAR for now

# Construct the CREATE TABLE statement
create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(f'{col} {typ}' for col, typ in zip(header, types))})"

# Execute the CREATE TABLE statement
cur.execute(create_table_sql)
conn.commit()

# Close the database connection
cur.close()
conn.close()
       

In [69]:
# Begin a new transaction
cur = conn.cursor()
conn.autocommit = False

try:
    # Open the CSV file
    with open(csv_file_path, 'r', newline='',encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        header = next(csvreader)  # Get the header row
        columns = ', '.join(header)  # Join column names with commas
        placeholders = ', '.join(['%s'] * len(header))  # Create placeholders for the values

        query = f"INSERT INTO ProductRecommendation ({columns}) VALUES ({placeholders})"

        for row in csvreader:
            cur.execute(query, row)

    # Commit the transaction
    conn.commit()
except Exception as e:
    # Roll back the transaction in case of any error
    conn.rollback()
    print(f"Error: {e}")

# Close the cursor and the connection
cur.close()
conn.close()